In [2]:
import sys
import pandas as pd
import s3fs
from time import time
import subprocess
sys.path.append('../src')
import cb_utils

pd.options.display.max_columns = 500
con = cb_utils.get_engine()

%load_ext autoreload
%autoreload 2

In [2]:
s3 = s3fs.S3FileSystem(s3_additional_kwargs={'ServerSideEncryption': 'AES256'})


### Declare variables

In [4]:
# configuration
mco_id = 1
file_name = 'uhc_20_10_29_tn_functional_assessment.txt'
dest_table_name = 'test_2020_10_29'
input_batch_descr = 'test descr'
input_descr = '...'
input_type = '...'
input_data_type = '...'
use_cache = False
path = 's3://cb-analytics-us-east-2-prd/' + file_name

In [20]:
s = time()
# df = pd.read_csv('s3://cb-analytics-us-east-2-prd/' + file_name)

chunk_size = 4096
i = 0
offset=0
chunks = []
returned = chunk_size

while returned != 0:
    chunk = s3.read(path, offset=offset, length=chunk_size, delimiter=b'\n')
    returned = len(chunk)
    chunks.append(chunk)
    offset +=returned
    if i % 1000 == 0:
        print(f'Processed: {offset} bytes in {time() - s:.2f}')
    i += 1
    
e = time()
cb_utils.printt(e-s)

Processed: 4116bytes in 0.19
Processed: 4218926bytes in 88.26
Processed: 8438691bytes in 168.94
Processed: 12656217bytes in 249.57
Processed: 16871581bytes in 332.57
Processed: 21090111bytes in 409.76
Processed: 25312172bytes in 488.41
Processed: 29532072bytes in 571.32
Processed: 33750475bytes in 649.69
Processed: 37970354bytes in 727.40
Processed: 42189548bytes in 804.97
Processed: 46414235bytes in 889.83
Processed: 50633506bytes in 971.19
Processed: 54849447bytes in 1048.31
Processed: 59065202bytes in 1126.03
Processed: 63289073bytes in 1211.64
Processed: 67511321bytes in 1293.53
Processed: 71734836bytes in 1368.75
Processed: 75952591bytes in 1450.48
Processed: 80175380bytes in 1539.16
Processed: 84398778bytes in 1616.74
Processed: 88622638bytes in 1695.25
Processed: 92844083bytes in 1775.52
Processed: 97060710bytes in 1857.30
Processed: 101276571bytes in 1936.80
Processed: 105499758bytes in 2017.11
Processed: 109725854bytes in 2097.35
Processed: 113950346bytes in 2178.45


KeyboardInterrupt: 

In [1]:
# crashes kernel
with s3.open(path) as f:
    data = f.read()
    print(data[:10])

NameError: name 's3' is not defined

In [5]:
s = time()
process = subprocess.run(['aws', 's3', 'cp', path, '.'], 
                         stdout=subprocess.PIPE, 
                         universal_newlines=True)
e = time()
cb_utils.printt(e-s)

Elapsed Time: 9.82s


In [ ]:
# 1000+ times faster using cli

### CREATE batch_input_id

In [ ]:
conn = cb_utils.get_conn()
query = """
insert into cb.input_batches (mco_id, description)
select %s, %s
returning id;
"""
with conn.cursor() as cur:
    cur.execute(query, [ mco_id, input_batch_descr])
    result = cur.fetchone()
conn.commit()
conn.close()

batch_id = result[0]
batch_id

In [ ]:
conn = cb_utils.get_conn()
query = """
insert into cb.inputs (input_batch_id, file_name, description, type, table_name, data_type, mco_id)
select %s, %s, %s, %s, %s, %s, %s
returning id;
"""
with conn.cursor() as cur:
    cur.execute(query, [ batch_id, auth_table_name, input_descr, input_type, auth_table_name, input_data_type, mco_id])
    result = cur.fetchone()
conn.commit()
conn.close()

input_id = result[0]
input_id

### Insert into raw

In [ ]:
df.to_sql(dest_table_name, con, 'raw')